# FinBERT: Financial Sentiment Analysis using Language Models

## Table of Contents
- [Introduction and Setup](#introduction)
- [BERT Fine-Tuning on Single Gaudi Card](#bert-fine-tuning-on-single-gaudi-card)
- [Fine-Tuning on Multiple Gaudis](#fine-tuning-on-multiple-gaudis)


## Introduction

This notebook shows how to train [FinBERT](https://arxiv.org/abs/1908.10063) on a Gaudi system. FinBERT is built by further training the BERT language model on a dataset in the finance domain and thus fine-tuning it for financial sentiment classification. Here, we will demonstrain how to use the [Financial PhraseBank](https://arxiv.org/abs/1307.5336v2) dataset for this fine-tuning. You will see how easy and fast it is to train this model on a Gaudi system and how you can further accelerate it using multiple Gaudis. 

## Setup

This notebook runs on a Gaudi system with SynapseAI and Gaudi drivers installed. Please refer to [installation guide](https://docs.habana.ai/en/latest/Installation_Guide/index.html) for more information.
We also use the transformers library from Habana's [Model Reference](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/nlp/finetuning/huggingface/bert/#setup) repository.

In addition, you need to download and extract Financial PhraseBank dataset from [ResearchGrate](https://www.researchgate.net/publication/251231364_FinancialPhraseBank-v10).

### Install Dependencies

In [ ]:
%pip install numpy pandas scikit-learn datasets
!git clone -b 1.5.0 https://github.com/HabanaAI/Model-References.git
%pip install Model-References/PyTorch/nlp/finetuning/huggingface/bert/transformers/.

# BERT Fine-Tuning on Single Gaudi Card

## Import Modules

In [3]:
import pandas as pd 
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset

# transformers library from Habana's Model Reference repository tailored for Gaudi
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer 
from transformers import Trainer, TrainingArguments, TextClassificationPipeline

### Import and Load Habana-related Modules

In [4]:
from habana_frameworks.torch.utils.library_loader import load_habana_module
load_habana_module()

Loading Habana modules from /usr/local/lib/python3.8/dist-packages/habana_frameworks/torch/lib


## Load Input Data

Each data point in the Financial PhraseBank dataset consists of financial text and label corresponding to sentiment of the text. The sentiment labels are as follows: 0 for _neutral_, 1 for _positive_, and 2 for _negative_ sentiments.

The workflow for loading and preparing input data is shown in the following diagram:

![workflow.svg](workflow.svg)

In [15]:
df = pd.read_csv(
    'FinancialPhraseBank-v1.0/Sentences_50Agree.txt',
    sep='@',
    names=['sentence', 'label'],
    encoding = "ISO-8859-1")
df = df.dropna()
df['label'] = df['label'].map({"neutral": 0, "positive": 1, "negative": 2})
df.head()

,sentence,label
0,"According to Gran , the company has no plans t...",0
1,Technopolis plans to develop in stages an area...,0
2,The international electronic industry company ...,2
3,With the new production plant the company woul...,1
4,According to the company 's updated strategy f...,1


In [16]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)

dataset_train = Dataset.from_pandas(df_train, preserve_index=False)
dataset_val = Dataset.from_pandas(df_val, preserve_index=False)
dataset_test = Dataset.from_pandas(df_test, preserve_index=False)

## Download Model

BERT model card is available on [HuggingFace](https://huggingface.co/bert-large-uncased). `BertForSequenceClassification` adds classification/regression head on top of the base BERT model allowing us to use it for a classification task.

In [ ]:
id2label = {0: "neutral", 1: "positive", 2: "negative"}
model_name = 'bert-large-uncased'

bert_config = BertConfig.from_pretrained(model_name, id2label=id2label)
bert_model = BertForSequenceClassification.from_pretrained(model_name, config=bert_config)
bert_tokenizer = BertTokenizer.from_pretrained(model_name)

## Prepare Dataset

Data needs to be transformed to the format accepted by [HuggingFace Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) class with fields expected by our model:

In [ ]:
dataset_train = dataset_train.map(lambda e: bert_tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: bert_tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: bert_tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train = dataset_train.remove_columns(['sentence'])
dataset_val = dataset_val.remove_columns(['sentence'])
dataset_test = dataset_test.remove_columns(['sentence'])

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

## Train the Model

Transformers library version from Habana's [Model Reference](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/nlp/finetuning/huggingface/bert/#setup) repository is capable of training models on Gaudi cards with minimal code changes. There are a few new training arguments that are described [here](https://github.com/HabanaAI/Model-References/blob/master/PyTorch/nlp/finetuning/huggingface/bert/transformers/src/transformers/training_args.py). 

In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

In [37]:
args = TrainingArguments(
    output_dir='temp/',
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    save_strategy='no',
    logging_strategy='epoch',
    logging_dir='logs/',
    report_to='tensorboard',

    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    metric_for_best_model='accuracy',

    use_habana=True,                   # use Habana device
    use_lazy_mode=True,                # use Gaudi lazy mode
    use_fused_adam=True,               # used optimised version of Adam for Gaudi
    use_fused_clip_norm=True,          # use Habana's fused gradient norm clipping operator
)


PyTorch: setting up devices
Enable use habana
Single node run
Enable habana lazy mode


If you want to optimize the speed of training, you can use [Habana Mixed Precision](https://developer.habana.ai/tutorials/pytorch/pytorch-mixed-precision/). It requires you to specify which operations should run using bf16 vs. fp32 precision. Sample values can be taken from [Model Reference](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/nlp/finetuning/huggingface/bert) or [Habana page on HuggingFace](https://huggingface.co/Habana).

In [13]:
args = TrainingArguments(
    output_dir='temp/',
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    save_strategy='no',
    logging_strategy='epoch',
    logging_dir='logs/',
    report_to='tensorboard',

    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    metric_for_best_model='accuracy',

    use_habana=True,                   # use Habana device
    use_lazy_mode=True,                # use Gaudi lazy mode
    use_fused_adam=True,               # used optimised version of Adam for Gaudi
    use_fused_clip_norm=True,          # use Habana's fused gradient norm clipping operator

    # Optional model improvements
    hmp=True,                          # use Habana mixed precision
    hmp_bf16="hmp_bf16_ops",           # file with list of operations performed in bf16
    hmp_fp32="hmp_fp32_ops",           # file with list of operations performed in fp32
)

PyTorch: setting up devices
Enable use habana
Single node run
Enable habana lazy mode


In [38]:
trainer = Trainer(
    model=bert_model,                   # the instantiated 🤗 Transformers model to be trained
    args=args,                          # training arguments, defined above
    train_dataset=dataset_train,        # training dataset
    eval_dataset=dataset_val,           # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()  

Enabled lazy mode
***** Running training *****
  Num examples = 3924
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2455


Epoch,Training Loss,Validation Loss,Accuracy
1,0.210600,0.441805,0.803204
2,0.142600,0.504630,0.846682
3,0.062400,0.580143,0.851259
4,0.029300,0.650701,0.848970
5,0.014500,0.670802,0.851259


***** Running Evaluation *****
  Num examples = 437
  Batch size = 4
***** Running Evaluation *****
  Num examples = 437
  Batch size = 4
***** Running Evaluation *****
  Num examples = 437
  Batch size = 4
***** Running Evaluation *****
  Num examples = 437
  Batch size = 4
***** Running Evaluation *****
  Num examples = 437
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2455, training_loss=0.09190003381484395, metrics={'train_runtime': 501.3593, 'train_samples_per_second': 39.134, 'train_steps_per_second': 4.897, 'total_flos': 4571138781987840.0, 'train_loss': 0.09190003381484395, 'epoch': 5.0})

<!--- ![finnet_full_diagram.png](finnet_full_diagram.svg) -->

## Evaluate the Model

Prediction results on train and test data:

In [ ]:
trainer.predict(dataset_train, metric_key_prefix="train").metrics

***** Running Prediction *****
  Num examples = 3924
  Batch size = 4


{'train_loss': 0.01688225194811821,
 'train_accuracy': 0.996177370030581,
 'train_runtime': 67.0568,
 'train_samples_per_second': 58.518,
 'train_steps_per_second': 14.629}

In [ ]:
trainer.predict(dataset_test).metrics

***** Running Prediction *****
  Num examples = 485
  Batch size = 4


{'test_loss': 0.6058623194694519,
 'test_accuracy': 0.8515463917525773,
 'test_runtime': 7.8169,
 'test_samples_per_second': 62.045,
 'test_steps_per_second': 15.607}

## Inference Examples

Here, we run our trained FinBERT model to recognize the sentiments of a few sample statements taken from the New York Times:

In [36]:
pipe = TextClassificationPipeline(model=bert_model, tokenizer=bert_tokenizer)
pipe.device=torch.device('hpu')

print(pipe("Alabama Takes From the Poor and Gives to the Rich"))
print(pipe("Economists are predicting the highest rate of employment in 15 years"))
print(pipe("It’s Been a Poor Year So Far for Municipal Bonds"))
print(pipe("Lavish Money Laundering Schemes Exposed in Canada"))
print(pipe("Stocks edge lower as bank earnings add to concerns about the economy"))

[{'label': 'neutral', 'score': 0.9094224572181702}]
[{'label': 'positive', 'score': 0.9752092957496643}]
[{'label': 'negative', 'score': 0.8343048095703125}]
[{'label': 'neutral', 'score': 0.5730178952217102}]
[{'label': 'negative', 'score': 0.9138352274894714}]


# Fine-Tuning on Multiple Gaudis

The simplest and efficient way of training transformers models on multiple Gaudi cards is by using `mpirun` command. All code cells from this notebook were copied to [finbert.py](finbert.py) python file. In order to run training on multiple cards, please use commands:
```
export MASTER_ADDR="localhost"
export MASTER_PORT="12345"
mpirun -n 8 --bind-to core --map-by socket:PE=7 --rank-by core --report-bindings --allow-run-as-root python3 finbert.py

```
Additional examples on distributed training of transformer models can be found in [Model Reference](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/nlp/finetuning/huggingface/bert#multicard-training). 